In [106]:
import numpy as np
import scipy
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [62]:
def write_answer(filename, answer):
    with open(str(filename), 'w') as file:
        file.write(str(answer))

2. Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [55]:
with open('SMSSpamCollection.txt', encoding='UTF-8') as file:
    texts = file.readlines()

3. Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [56]:
labels = [0 if 'ham' in i else 1 for i in texts]

In [57]:
texts = [i.replace('ham', '') for i in texts]
texts = [i.replace('spam', '') for i in texts]
texts = [i.strip() for i in texts]

4. Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [58]:
vzr = CountVectorizer()
vecttexts = vzr.fit_transform(texts)

5. Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [59]:
pipeline = Pipeline([('count_vect', CountVectorizer()), ('log_reg', LogisticRegression())])

In [61]:
logreg = LogisticRegression()
cross_val_score(pipeline, texts, labels, cv=10, scoring='f1').mean()


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9315048054196826

In [66]:
write_answer('answer1.txt', round(0.9315048054196826, 1))

6. А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [71]:
predictlist = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
                      "FreeMsg: Txt: claim your reward of 3 hours talk time", 
                      "Have you visited the last lecture on physics?", 
                      "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
                      "Only 99$"]

In [80]:
pipeline = Pipeline([('count_vect', CountVectorizer()), ('log_reg', LogisticRegression())])
pipeline.fit(texts, labels)

Pipeline(memory=None,
     steps=[('count_vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [84]:
prediction = pipeline.predict(predictlist)

In [88]:
with open('answer2.txt', 'w') as f:
    ans2 = ' '.join(map(str, prediction))
    f.write(ans2)

7. Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [103]:
ans3 = []
for i in [[2,2], [3,3], [1,3]]:
    pipeline = Pipeline([('count_vect', CountVectorizer(ngram_range=(i))), ('log_reg', LogisticRegression())])
    ans = cross_val_score(pipeline, texts, labels, cv=10, scoring='f1').mean()
    print('Ngram range:', i)
    print('Cross-val-score f1: %.4f\n' % ans)
    ans3.append(round(ans,2))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Ngram range: [2, 2]
Cross-val-score f1: 0.8168



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Ngram range: [3, 3]
Cross-val-score f1: 0.7224



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Ngram range: [1, 3]
Cross-val-score f1: 0.9225



In [104]:
with open('answer3.txt', 'w') as f:
    answer3 = ' '.join(map(str, ans3))
    f.write(answer3)

In [105]:
answer3

'0.82 0.72 0.92'

8. Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.

In [113]:
ans4 = []
for i in [[2,2], [3,3], [1,3]]:
    CV = CountVectorizer(ngram_range=(i))
    vectortexts = CV.fit_transform(texts)
    NBCL = MultinomialNB()
    ans = cross_val_score(NBCL, vectortexts, labels, cv=10, scoring='f1').mean()
    print('Ngram range:', i)
    print('Cross-val-score f1: %.4f\n' % ans)
    ans4.append(round(ans,2))

Ngram range: [2, 2]
Cross-val-score f1: 0.6424

Ngram range: [3, 3]
Cross-val-score f1: 0.3768

Ngram range: [1, 3]
Cross-val-score f1: 0.8876



In [116]:
with open('answer4.txt', 'w') as f:
    answer4 = ' '.join(map(str, ans4))
    f.write(answer4)

9. Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [114]:
pipeline = Pipeline([('count_vect', TfidfVectorizer()), ('log_reg', LogisticRegression())])
cross_val_score(pipeline, texts, labels, cv=10, scoring='f1').mean()


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8747673456237856

In [117]:
write_answer('answer5.txt', '-1')